Imports

In [1]:
!pip install tensorflow
!pip install tflearn
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [80]:
import cv2
import numpy as np
import pandas as pd
import os
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img, ImageDataGenerator
from sklearn.model_selection import train_test_split

import tensorflow
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

Define each training and testing dataset paths

In [85]:
modelName = 'DogsvCats.model'

trainingDir = '/home/nour/Documents/University/4740/dogs-vs-cats/train'
testingDir = '/home/nour/Documents/University/4740/dogs-vs-cats/test1'

fileNamesTraining = os.listdir(trainingDir)
fileNamesTesting = os.listdir(testingDir)


IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscats.model'


### FUNCTIONS TO BREAKDOWN THE FILES

In [59]:

# get the animal in the image from the training dataset
def renameImages(image):
    
    word_label = image.split('.')[-3] # split to get the animal, 
    
    # based on competition instructions, one hot encode the values
    # [x, y] x = 1 for cat, y = 1 for dog
    if word_label == 'cat': 
        return [1,0]
    
    elif word_label == 'dog': 
        return [0,1]
    
# process each image and save the labels into the trainingData array
def generateTrainingDataset():
    
    trainingData = [] # array to save all the image labels from trainingData
    
    for file in fileNamesTraining: # for each file:
        
        newName = renameImages(file) # rename each file
        
        path = os.path.join(trainingDir,file)
        
        image = cv2.imread(path,cv2.IMREAD_GRAYSCALE) # make each image uniform
        image = cv2.resize(image, (50, 50))
        
        # add the image feature and its corresponding name to the array
        trainingData.append([np.array(file),np.array(newName)])
        
    shuffle(trainingData) # shuffle the dataset to randomize the labels
    np.save('trainingData.npy', trainingData)
    return trainingData

def generatingTestingDataset():
    
    testingData = []
    
    for file in fileNamesTesting: # for each image in the testing dataset:
        
        path = os.path.join(testingDir, file) # generate a new path name, concatenate 
        
        imageNum = file.split('.')[0] # get the image number
        
        image = cv2.imread(path,cv2.IMREAD_GRAYSCALE) # 
        image = cv2.resize(file, (50, 50))
        
        testingData.append([np.array(file), imageNum])
        
    shuffle(testingData)
    np.save('testingData.npy', testingData)
    return testingData

In [73]:
trainingData = generateTrainingDataset()

In [61]:
trainingData

[[array('dog.7839.jpg', dtype='<U12'), array([0, 1])],
 [array('dog.4638.jpg', dtype='<U12'), array([0, 1])],
 [array('cat.2714.jpg', dtype='<U12'), array([1, 0])],
 [array('dog.760.jpg', dtype='<U11'), array([0, 1])],
 [array('cat.6296.jpg', dtype='<U12'), array([1, 0])],
 [array('dog.8008.jpg', dtype='<U12'), array([0, 1])],
 [array('dog.8156.jpg', dtype='<U12'), array([0, 1])],
 [array('dog.10085.jpg', dtype='<U13'), array([0, 1])],
 [array('cat.11194.jpg', dtype='<U13'), array([1, 0])],
 [array('cat.2429.jpg', dtype='<U12'), array([1, 0])],
 [array('dog.7604.jpg', dtype='<U12'), array([0, 1])],
 [array('cat.8591.jpg', dtype='<U12'), array([1, 0])],
 [array('cat.1239.jpg', dtype='<U12'), array([1, 0])],
 [array('dog.12137.jpg', dtype='<U13'), array([0, 1])],
 [array('cat.6008.jpg', dtype='<U12'), array([1, 0])],
 [array('dog.10569.jpg', dtype='<U13'), array([0, 1])],
 [array('cat.1054.jpg', dtype='<U12'), array([1, 0])],
 [array('dog.12002.jpg', dtype='<U13'), array([0, 1])],
 [arra

## Create a simple 2-Layered CNN

In [62]:
convnet = input_data(shape=[None, 50, 50, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate = 1e-3, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


train = trainingData[:-500]
test = trainingData[-500:]


In [77]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch = 3, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id = MODEL_NAME)

ValueError: cannot reshape array of size 24500 into shape (50,50,1)

## Larger CNN

In [83]:
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

In [84]:
train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch = 10, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id = MODEL_NAME)

---------------------------------
Run id: dogsvscats-0.001-2conv-basic.model
Log directory: log/
INFO:tensorflow:Summary name Accuracy/Adam_0_0_0 (raw) is illegal; using Accuracy/Adam_0_0_0__raw_ instead.
INFO:tensorflow:Summary name Accuracy_1/Adam_1_0_0 (raw) is illegal; using Accuracy_1/Adam_1_0_0__raw_ instead.
INFO:tensorflow:Summary name Accuracy_2/Adam_0_1_0 (raw) is illegal; using Accuracy_2/Adam_0_1_0__raw_ instead.
INFO:tensorflow:Summary name Accuracy_3/Adam_1_1_0 (raw) is illegal; using Accuracy_3/Adam_1_1_0__raw_ instead.
INFO:tensorflow:Summary name Accuracy_4/Adam_0_0_1 (raw) is illegal; using Accuracy_4/Adam_0_0_1__raw_ instead.
INFO:tensorflow:Summary name Accuracy_5/Adam_1_0_1 (raw) is illegal; using Accuracy_5/Adam_1_0_1__raw_ instead.
INFO:tensorflow:Summary name Accuracy_6/Adam_0_1_1 (raw) is illegal; using Accuracy_6/Adam_0_1_1__raw_ instead.
INFO:tensorflow:Summary name Accuracy_7/Adam_1_1_1 (raw) is illegal; using Accuracy_7/Adam_1_1_1__raw_ instead.
INFO:tensor

InvalidArgumentError: You must feed a value for placeholder tensor 'input_1/X' with dtype float and shape [?,50,50,1]
	 [[node input_1/X (defined at home/nour/.local/lib/python3.9/site-packages/tflearn/layers/core.py:81) ]]

Original stack trace for 'input_1/X':
  File "usr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "usr/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "home/nour/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "home/nour/.local/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "home/nour/.local/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
    self.io_loop.start()
  File "home/nour/.local/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "usr/l

In [65]:
fig = plt.figure()
testData = generatingTestingDataset()

for num,data in enumerate(testData[:12]):
    
    imgNum = data[1]
    imgData = data[0]
    
    y = fig.add_subplot(3, 4, num + 1)
    
    orig = img_data
    data = img_data.reshape(50, 50,1)
    
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: str_label='Dog'
    else: str_label='Cat'
        
    y.imshow(orig,cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
    
plt.show()

TypeError: Expected Ptr<cv::UMat> for argument 'src'

<Figure size 432x288 with 0 Axes>